In [ ]:
import pandas as pd

# 정규화
def kor(text):
    한글 = '[^가-힣ㄱ-ㅎㅏ-ㅢ]'
    result = re.sub(한글, '', str(text))
    result.strip()
    return result

# 맞춤법 검사
from hanspell import spell_checker

# 맞춤법
import requests, ast
from bs4 import BeautifulSoup as bs

# hanspell 실행 관련 문제가 많이 발생하기 때문에 hanspell에서 구현한 방법 중 일부를 차용하여 함수를 구성합니다.
def spell_check(text):
    result = None
    with requests.Session() as req:
        base_url= 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'
        payload = {
            '_callback':'window.__jindo2_callback._spellingCheck_0',
            'q': text
        }
        headers = {
            'referer': 'https://search.naver.com/',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
        }
        result = req.get(base_url, params=payload, headers = headers).text
        _from = result.index('(')+1
        _to = len(result) - (result[::-1].index(')')) - 1
        result = ast.literal_eval(result[_from:_to])['message']['result']
    output = {k:bs(v, 'lxml').text for k,v in result.items() if isinstance(v, str)}
    return output['html']

### 맞춤법 + 예외처리 ###
def correction(text, exceptions=[]):
    dic = {excp:f'<<<{i}>>>' for i,excp in enumerate(exceptions)}
    inv_dic = {v:k for k,v in dic.items()}
    for k,v in dic.items():
        text = text.replace(k,v)
        # 눈<은>, <은>의 가격
        #'(?<=[가-힣])은'
        #text = re.sub(k,v,text)
    text = spell_check(text)
    for k,v in inv_dic.items():
        text = text.replace(k,v)
    return text


##### 반복되는 문자 제거 ######
from soynlp.normalizer import *

def repeat(text):
    new_text = emoticon_normalize(text, num_repeats = 2)
    return new_text

##### 입력 오타 수정기 ######

def jamo_merge(jamolist):
    consonant = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    vowel = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonant = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ',
                       'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    a,b,c = jamolist
    if (a in consonant) and (b in vowel) and (c in final_consonant):
        c, v, f_c = [ _list.index(j) for _list, j in zip([consonant,vowel,final_consonant],jamolist)]
        return chr(f_c + 588 * c + 28 * v + ord('가'))
    else:
        result = ' '.join(jamolist)
        result = ' ' if result.strip()=='' else result.strip()
        return result

def jamo_split(char):
    consonant = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    vowel = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    final_consonant = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ',
                       'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    base = ord(char) - ord('가')
    c = base // 588
    v = (base - 588 * c) // 28
    f_c = base - 588 * c - 28 * v
    return [consonant[c], vowel[v], final_consonant[f_c]]


## 이거 찐 ##
def jamo_correction(text):
    consonant,vowel = (12593, 12622),(12623,12686) #각각 자음, 모음의 범위
    merged = (44032, 55203) #<가-힣>까지의 조합된 한글의 범위
    isin = lambda c, nrange: c >= nrange[0] and c <= nrange[1]  
    result = [[]]
    text = text+' '
    for i in range(len(text.strip())):
        if not isin(ord(text[i]), merged):
            if isin(ord(text[i]), vowel) and len(result[-1])==1:
                result[-1].append(text[i])
            elif isin(ord(text[i]), consonant) and result[-1][-1]==' ' and len(result[-1])==3:
                if isin(ord(text[i+1]), merged):
                    result[-1][-1] = text[i]
                else:
                    result.append([text[i]])
            else:
                result.append([])
                result[-1].append(text[i])
        else:
            result.append(jamo_split(text[i]))
    result = result[1:]
    output = [[' ' for i in range(3)] for j in range(len(result))]
    for i in range(len(output)):
        output[i][:len(result[i])]= result[i]        
    output = list(map(jamo_merge, output))
    return ''.join(output)


##### 고유 명사 예외처리 ######

# import pandas as pd
# path = './NIADic.xlsx' 
# df_new = pd.read_excel(path)
# df_new1 = df_new.loc[(df_new.category=='people_names')]
# df_new2 = df_new.loc[(df_new.category=='brand_name')]
# df_new3 = df_new.loc[(df_new.category=='proper_noun')]
# total = pd.concat([df_new1,df_new2,df_new3], ignore_index=True, sort=False)




# dic_result = {'terms': total['term'].values.tolist()}
# with open('./data/proper_noun.json', 'w', encoding='utf-8') as f:
#     json.dump(dic_result, f)



import json
import re
import ast
import requests
from itertools import *
from bs4 import BeautifulSoup as bs
from hanspell import spell_checker
from soynlp.normalizer import *

class Corrector(object):
    def __init__(self):
        
        with open('./고유명사.json', 'r', encoding='utf-8') as r:
            self.proper_nouns = json.load(r)['terms']
        
        self.consonant = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        self.vowel = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
        self.final_consonant = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ',
                           'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    def remove_repeat(self, text, repeat=2):
        result = [[]]
        for t in text:
            if (t in result[-1]) and not (ord('가')<=ord(t)<=ord('힣')):
                result[-1].append(t)
            else:
                result.append([t])
        result = list(chain(*map(lambda x: x[:repeat], result[1:])))
        return ''.join(result)
        
    def merge_hangul(self, text):
        consonant,vowel = (12593, 12622),(12623,12686) #각각 자음, 모음의 범위
        merged = (44032, 55203) #<가-힣>까지의 조합된 한글의 범위
        isin = lambda c, nrange: c >= nrange[0] and c <= nrange[1]  
        result = [[]]
        text = text+' '
        for i in range(len(text.strip())):
            if not isin(ord(text[i]), merged):
                if isin(ord(text[i]), vowel) and len(result[-1])==1:
                    result[-1].append(text[i])
                elif isin(ord(text[i]), consonant) and result[-1][-1]==' ' and len(result[-1])==3:
                    if isin(ord(text[i+1]), merged):
                        result[-1][-1] = text[i]
                    else:
                        result.append([text[i]])
                else:
                    result.append([])
                    result[-1].append(text[i])
            else:
                result.append(self._jamo_split(text[i]))
        result = result[1:]
        output = [[' ' for i in range(3)] for j in range(len(result))]
        for i in range(len(output)):
            output[i][:len(result[i])]= result[i]        
        output = list(map(self._jamo_merge, output))
        return ''.join(output)
    
    def _jamo_split(self, char):
        base = ord(char) - ord('가')
        c = base // 588
        v = (base - 588 * c) // 28
        f_c = base - 588 * c - 28 * v
        return [self.consonant[c], self.vowel[v], self.final_consonant[f_c]]
    
    def _jamo_merge(self, jamolist):
        a,b,c = jamolist
        if (a in self.consonant) and (b in self.vowel) and (c in self.final_consonant):
            c, v, f_c = [ _list.index(j) for _list, j in zip([self.consonant,self.vowel,self.final_consonant],jamolist)]
            return chr(f_c + 588 * c + 28 * v + ord('가'))
        else:
            result = ' '.join(jamolist)
            result = ' ' if result.strip()=='' else result.strip()
            return result
    
    def spell_check(self,text):
        result = None
        with requests.Session() as req:
            base_url= 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'
            payload = {
                '_callback':'window.__jindo2_callback._spellingCheck_0',
                'q': text
            }
            headers = {
                'referer': 'https://search.naver.com/',
                'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
            }
            result = req.get(base_url, params=payload, headers = headers).text
            _from = result.index('(')+1
            _to = len(result) - (result[::-1].index(')')) - 1
            result = ast.literal_eval(result[_from:_to])['message']['result']
        output = {k:bs(v, 'lxml').text for k,v in result.items() if isinstance(v, str)}
        return output


    
    def masking_exception(self, text, exceptions=[]):
        masked = []
        origins = []
        tokens = self.tokenize(text)
        for token in tokens:
            if token in exceptions:
                origins.append(token)
                token = '<MASK>'
            masked.append(token)
        return masked, origins

    def tokenize(self, text):
        output_tokens = []
        for token in text.split():
            sub_tokens = self.sub_tokenize(token)
            output_tokens += sub_tokens
            output_tokens.append(' ')
        return output_tokens

    def sub_tokenize(self, text):
        post_position = ['이','가','을','를','는','로','의', '입니다','입니다.', '습니다'] # 더 많이 있지만 예시를 위해 이 정도로 제한
        expression = '({})$'.format('|'.join(post_position))
        sub_tokens = re.split(expression,text)
        sub_tokens = [t for t in sub_tokens if t!='']
        return sub_tokens

    def correct(self, text, n_repeat=1):
        text = self.remove_repeat(text, n_repeat)
        text = self.merge_hangul(text)
        textlist, origin_words = self.masking_exception(text, self.proper_nouns)
        text = ' '.join(textlist)
        text = self.spell_check(text)['html']
        text = text.split()
        for word in origin_words:
            for i,token in enumerate(text):
                if token.find('<MASK>')!=-1:
                    strt = token.index('<MASK>')
                    text[i] = token[:strt]+word+token[strt+len('<MASK>'):]
        return ' '.join(text)

corrector = Corrector()
# corrector.correct('잇몸이 튼튼해지는 듯요 입안도 개운하고요')


from itertools import chain

def remove_repeat(text, repeat=1):
    result = [[]]
    for t in text:
        if (t in result[-1]) and not (ord('가')<=ord(t)<=ord('힣')):
            result[-1].append(t)
        else:
            result.append([t])
    result = list(chain(*map(lambda x: x[:repeat], result[1:])))
    return ''.join(result)

In [ ]:
review = pd.read_csv('./치약 리뷰 총합.csv')
review['수정'] = review['내용'].apply(lambda x:kor(x))

review

,내용,날짜,수정
0,잇몸이 튼튼해지는 듯요. 입안도 개운하고요.~,NaN,잇몸이튼튼해지는듯요입안도개운하고요
1,자주쓰는 치약이에요 좋아요,NaN,자주쓰는치약이에요좋아요
2,늘 사용하는 제품 저렴하게 구매했네요\n상쾌하고 압안이 개운해요,NaN,늘사용하는제품저렴하게구매했네요상쾌하고압안이개운해요
3,아침에 일어나면 입냄새가 나지 안아서 좋아요 텁텁함도 없구요,NaN,아침에일어나면입냄새가나지안아서좋아요텁텁함도없구요
4,초2 아들도 이 치약만 써요.\n양치하고나서 개운해요,NaN,초아들도이치약만써요양치하고나서개운해요
...,...,...,...
29978,사업자 반값구매 혜택 좋네요,NaN,사업자반값구매혜택좋네요
29979,잘쓸개요 충치야 생기지마라,NaN,잘쓸개요충치야생기지마라
29980,배송빠르고 상품조아요,NaN,배송빠르고상품조아요
29981,치약 한참 쓰겠네요...,NaN,치약한참쓰겠네요


In [29]:
def aaa(text):
    return text


In [6]:
gb = []
for i in range(0, 29883):
    try:
        gb.append(correction(review['수정'].iloc[i], exceptions = []))
    except:
        gb.append('')
gb


In [ ]:
review.loc[:, '수정2'] = gb

In [ ]:
review['수정2'] = review['수정2'].apply(lambda x:remove_repeat(x, repeat=1))

In [ ]:
review['수정2'] = review['수정2'].apply(lambda x:jamo_correction(x))

In [ ]:
review.to_csv('./문제.csv', encoding = 'utf-8-sig')